<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/news/notebooks/example01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install func2pipe==0.3.6

  Found existing installation: func2pipe 0.3.5
    Uninstalling func2pipe-0.3.5:
      Successfully uninstalled func2pipe-0.3.5


In [ ]:
import func2pipe.func2pipe as fp
dir(fp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'convertToPipeFunc',
 'convertToPipeFunc2',
 'convertToPipeFuncFull',
 'createPipe',
 'createSub',
 'partial',
 'pipeit',
 'wraps']

In [ ]:
#from func2pipe import createPipe, pipeit
#import func2pipe as fp

#print(fp.func2pipe)
@fp.pipeit()
def r(item):
  return item + 2

operator = fp.createPipe(
    r(),
    list
)

print(operator([0, 1, 2]))

[2, 3, 4]
